In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import os
import pickle
import pandas as pd
import json

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(5)
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
wait = WebDriverWait(driver, 5, ignored_exceptions = ignored_exceptions)

In [2]:
seasons = ['http://www.oddsportal.com/soccer/england/premier-league-2010-2011/results',
          'http://www.oddsportal.com/soccer/england/premier-league-2011-2012/results',
          'http://www.oddsportal.com/soccer/england/premier-league-2012-2013/results',
          'http://www.oddsportal.com/soccer/england/premier-league-2013-2014/results',
          'http://www.oddsportal.com/soccer/england/premier-league-2014-2015/results',
          'http://www.oddsportal.com/soccer/england/premier-league-2015-2016/results',
          'http://www.oddsportal.com/soccer/england/premier-league-2016-2017/results',
          'http://www.oddsportal.com/soccer/england/premier-league/results/']

In [16]:
def getOdds(season):
    driver.get(season)
    df = pd.DataFrame(columns = ['homeTeam', 'awayTeam', 'homeOdds', 'drawOdds', 'awayOdds'])
    pageNums = driver.find_element_by_id('pagination').text
    pages = [x for x in pageNums if x.isdigit()]

    for i in pages:
        url = "%s/#/page/%s" % (season, i)
        driver.get(url)
        time.sleep(3)
        #odds
        odds = driver.find_elements_by_class_name('odds-nowrp')
        oddsList = []
        for i in odds:
            oddsList.append(i.text)
        oddsList = [oddsList[i:i+3] for i in range(0, len(oddsList), 3)]
        #teams
        time.sleep(3)
        teams = driver.find_elements_by_css_selector('.name.table-participant')
        teamsList = []
        for i in teams:
            teamsList.append(i.text)

        #rearranging data
        homeTeam = [x.split(" -")[0] for x in teamsList]
        awayTeam = [x.split("- ")[1] for x in teamsList]
        homeOdds = [x[0] for x in oddsList]
        drawOdds = [x[1] for x in oddsList]
        awayOdds = [x[2] for x in oddsList]

        #convert to df and append to master df
        dftemp = pd.DataFrame({'homeTeam' : homeTeam, 'awayTeam': awayTeam, 'homeOdds': homeOdds,
                  'drawOdds': drawOdds, 'awayOdds': awayOdds})
        df = df.append(dftemp, ignore_index = True)
    if len(season) >= 73:
        df.to_pickle("odds%s.pkl" % (season[-17:-8].replace("-", "")))
    else:
        df.to_pickle("odds20172018.pkl")

In [18]:
for season in seasons:
    getOdds(season)